# Data preprocessing

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.engine import data_adapter
from tensorflow.keras import backend as K
import os, sys
os.mkdir('/kaggle/working/ckpts/')
sys.path.append('../input/boneage-ckpts')
sys.path.append('../input/hdaproject2021-codes/')
sys.path.append('../input/temppp/')

In [2]:
train_csv_directory = '../input/hdadataset2021/Bone Age Datasets/Bone Age Training Set/'
train_directory = '../input/hdadataset2021/Bone Age Datasets/Bone Age Training Set/boneage-training-dataset/'
df_train = pd.read_csv(train_csv_directory + 'train.csv')

val_csv_directory = '../input/hdadataset2021/'
val_directory = '../input/hdadataset2021/boneage-validation-dataset/'
df_val = pd.read_csv(val_csv_directory + 'Validation Dataset.csv')

In [3]:
ids = df_train['id'].values.astype(str)
labels = df_train['male'].values.astype(float)
ages = df_train['boneage'].values.astype(float)
train_ds = tf.data.Dataset.from_tensor_slices((ids, labels, ages))


ids = df_val['Image ID'].values.astype(str)
labels = df_val['male'].values.astype(float)
ages = df_val['Bone Age (months)'].values.astype(float)
val_ds = tf.data.Dataset.from_tensor_slices((ids, labels, ages))

IMG_SIZE = (300, 300, 1)
BATCH_SIZE = 32

In [4]:
def read_train_image(image_file, label, age):
    image = tf.io.read_file(train_directory + image_file+ '.png')
    image = tf.io.decode_png(image, channels=1)
    return image, [label], [age] #lists are necessary for training the model (otherwise dims are unknown)

def read_val_image(image_file, label, age):
    image = tf.io.read_file(val_directory + image_file + '.png')
    image = tf.io.decode_png(image, channels=1)
    return image, [label], [age] #lists are necessary for training the model (otherwise dims are unknown)

resize_layer = tf.keras.layers.experimental.preprocessing.Resizing(height=IMG_SIZE[0], width=IMG_SIZE[1])
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)
zoom_layer = tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(0.2,-0.3),
                                                                   width_factor=(0.2,-0.3),
                                                                  fill_mode='constant')
flip_layer = tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal')

def augment(image, label, age):
    # data augmentation here
    image = zoom_layer(image)
    image = flip_layer(image)
    return image, label, age

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE #tf.data.AUTOTUNE
train_ds = train_ds.map(read_train_image).map(lambda image,label,age: (normalization_layer((resize_layer(image))), label, age)).cache('../input/hdaproject2021-codes/train_cache_300')
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).map(augment,num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(read_val_image).map(lambda image,label,age: (normalization_layer(resize_layer(image)), label, age)).cache('../input/hdaproject2021-codes/val_cache_300')
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE).batch(BATCH_SIZE)

# Boneage Model NO GENDER INFO

In [6]:
from InceptionV3_2_initializer import InceptionV3

class BoneAgeInception(tf.keras.Model):  
    def __init__(self, gender=True, beta=1, initializer=tf.keras.initializers.HeNormal()):
        super(BoneAgeInception, self).__init__()
        
        model = InceptionV3(
                                include_top=False,
                                weights=None,
                                input_shape=IMG_SIZE,
                                beta=beta, initializer=initializer)
                
        
            
        model_global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)

        overall_output = tf.keras.layers.Dense(1,activation='relu', kernel_regularizer='l2', kernel_initializer=initializer)(model_global_avg)

        self.model = tf.keras.Model(inputs=model.input, outputs = overall_output)
                
        
        
    def call(self, x, training=False):
        imgs, labels = x
        
        y = self.model(imgs,training)
       
        return y

    def train_step(self, data):
        
        data = data_adapter.expand_1d(data)
        imgs, labels, ages = data
        
        with tf.GradientTape() as tape:
            y_pred = self((imgs, labels), training=True)  # Forward pass

            loss = self.compiled_loss(ages, y_pred)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(ages, y_pred)
        _dict = {m.name: m.result() for m in self.metrics}
        
        return _dict

    def test_step(self, data):
        
        data = data_adapter.expand_1d(data)
        imgs, labels, ages = data

        y_pred = self((imgs, labels), training=False)  # Forward pass

        loss = self.compiled_loss(ages, y_pred)

        self.compiled_metrics.update_state(ages, y_pred)
        _dict = {m.name: m.result() for m in self.metrics}
        
        return _dict
    
    def summary(self):
        print(self.model.summary())

In [7]:
initializer=tf.keras.initializers.HeUniform()
bonageinception_mod = BoneAgeInception(beta=1.4, initializer=initializer)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath='./best_val_inception_no_gender_80ep.h5', monitor='val_mae', verbose=1, save_best_only=True,
                save_weights_only=True, mode='min')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    elif epoch < 60:
        return 1e-3
    else:
        return 1e-4

lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
metric = tf.keras.metrics.MeanAbsoluteError()
bonageinception_mod.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

bonageinception_mod.fit(train_ds, validation_data=val_ds,callbacks = [ckpt_callback,lr_scheduler_callback],
                    epochs=80, verbose=2)

Epoch 1/80

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
395/395 - 257s - loss: 1885.2074 - mae: 34.5708 - val_loss: 3409.0725 - val_mae: 46.0386

Epoch 00001: val_mae improved from inf to 46.03860, saving model to ./best_val_inception_no_gender_80ep.h5
Epoch 2/80

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
395/395 - 234s - loss: 1661.4110 - mae: 33.1002 - val_loss: 15969.5312 - val_mae: 109.9022

Epoch 00002: val_mae did not improve from 46.03860
Epoch 3/80

Epoch 00003: LearningRateScheduler reducing learning rate to 0.009999999776482582.
395/395 - 234s - loss: 1594.2401 - mae: 32.1354 - val_loss: 3256.6494 - val_mae: 44.9217

Epoch 00003: val_mae improved from 46.03860 to 44.92170, saving model to ./best_val_inception_no_gender_80ep.h5
Epoch 4/80

Epoch 00004: LearningRateScheduler reducing learning rate to 0.009999999776482582.
395/395 - 234s - loss: 1570.8524 - mae: 31.9505 - val_loss: 3145.1887 - val_ma

### test

In [8]:
test_directory = '../input/hdadataset2021/Bone Age Datasets/Bone Age Test Set/Test Set Images/'

df_test = pd.read_csv('../input/hdaproject2021-codes/test.csv')

ids = df_test['id'].values.astype(str)
labels = (df_test['Sex'].values == 'M').astype(float)

# replace ',' with '.' and convert to float
ages_str = df_test['boneage'].values
ages = np.empty_like(ages_str)
for i, item in enumerate(ages_str):
    replaced = item.replace(',','.')
    replaced_float = float(replaced)
    ages[i] = replaced_float
ages = ages.astype(float)

test_ds = tf.data.Dataset.from_tensor_slices((ids, labels, ages))

def read_test_image(image_file, label, age):
    image = tf.io.read_file(test_directory + image_file + '.png')
    image = tf.io.decode_png(image, channels=1)
    return image, [label], [age] #lists are necessary for training the model (otherwise dims are unknown)

test_ds = test_ds.map(read_test_image).map(lambda image,label,age: (normalization_layer(resize_layer(image)), label, age)).cache()
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

test_iterator = test_ds.as_numpy_iterator()
N = 200 #number of samples
test_images = np.empty(shape=(N, IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)
test_labels = np.empty(shape=(N,), dtype=np.float32)
test_ages = np.empty(shape=(N,), dtype=np.float32)
for i, item in enumerate(test_iterator):
    test_images[i] = item[0]
    test_labels[i] = item[1]
    test_ages[i] = item[2]
    
print(test_images.shape)
print(test_labels.shape)
print(test_ages.shape)

(200, 300, 300, 1)
(200,)
(200,)


In [9]:
bonageinception_mod.load_weights('./best_val_inception_no_gender_80ep.h5')

In [10]:
bonageinception_mod_pred = bonageinception_mod.predict((test_images, test_labels))

In [11]:
np.mean(np.abs(bonageinception_mod_pred.squeeze() - test_ages.squeeze()))

8.885149

In [12]:
bonageinception_mod.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 44) 396         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 44) 132         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 44) 0           batch_normalization[0][0]        
______________________________________________________________________________________________